In [0]:
%sh
pip install nltk
pip install click
pip install html2text
pip install lxml
pip install markdown2
pip install numpy
pip install pandas
pip install python-dateutil
pip install pytz
pip install regex
pip install scikit-learn
pip install scipy
pip install six
pip install sklearn
pip install soupsieve
pip install threadpoolctl
pip install tqdm
pip install bs4
pip install beautifulsoup4
pip install --upgrade pip
python -m nltk.downloader all

/usr/lib/python3.8/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /roo

In [0]:
import nltk
import re
import pandas
from pandas import DataFrame
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer
from joblib import Parallel
from joblib import delayed
from sklearn.utils import resample
from sklearn.utils.validation import check_is_fitted
from sklearn.base import BaseEstimator, clone
import logging

import sqlite3
from sqlite3 import Error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [0]:
class _ConstantPredictor(BaseEstimator):

    def fit(self, X, y):
        self.y_ = y
        return self

    def predict(self, X):
        check_is_fitted(self, 'y_')

        return np.repeat(self.y_, X.shape[0])

    def decision_function(self, X):
        check_is_fitted(self, 'y_')

        return np.repeat(self.y_, X.shape[0])

    def predict_proba(self, X):
        check_is_fitted(self, 'y_')

        return np.repeat([np.hstack([1 - self.y_, self.y_])],
                         X.shape[0], axis=0)


def _fit_binary(estimator, X, y, classes=None):
    """Fit a single binary estimator."""
    unique_y = np.unique(y)
    if len(unique_y) == 1:
        if classes is not None:
            if y[0] == -1:
                c = 0
            else:
                c = y[0]

        estimator = _ConstantPredictor().fit(X, unique_y)
    else:
        estimator = clone(estimator)
        estimator.fit(X, y)
    return estimator

class OneVsRestClassifierBalance(OneVsRestClassifier):

    def fit(self, X, y):
        self.label_binarizer_ = LabelBinarizer(sparse_output=True)
        Y = self.label_binarizer_.fit_transform(y)
        Y = Y.tocsc()
        self.classes_ = self.label_binarizer_.classes_
        totalIns = Y.shape[0]
        XBal = []
        YBal = []
        for i in range(len(self.label_binarizer_.classes_)):
            if len(y.shape) > 1:
                # Matrix
                curIdxs = Y[:, i].nonzero()[0]
            else:
                curIdxs = Y.nonzero()[0]
            baseX = X[curIdxs, :]
            if len(y.shape) > 1:
                # Matrix
                baseY = y[curIdxs, :]
            else:
                # array, e.g. due to testing classifier performance for single label prediction
                baseY = y[curIdxs]
            tempX = X
            tempY = y
            imbalancedIns = baseX.shape[0]
            numDup = totalIns / imbalancedIns - 1
            for j in range(int(numDup)):
                tempX = np.vstack((tempX, baseX))
                if len(y.shape) > 1:
                    tempY = np.vstack((tempY, baseY))
                else:
                    tempY = np.concatenate((tempY, baseY))
            numAdd = totalIns % imbalancedIns
            tempX = np.vstack((tempX, resample(baseX, n_samples=numAdd, random_state=0)))
            if len(y.shape) > 1:
                tempY = np.vstack((tempY, resample(baseY, n_samples=numAdd, random_state=0)))
            else:
                tempY = np.concatenate((tempY, resample(baseY, n_samples=numAdd, random_state=0)))
            XBal.append(tempX)
            if len(y.shape) > 1:
                YBal.append(tempY[:, i])
            else:
                YBal.append(tempY)
        self.estimators_ = Parallel(n_jobs=self.n_jobs)(delayed(_fit_binary)(
            self.estimator, XBal[i], YBal[i], classes=[
                "not %s" % self.label_binarizer_.classes_[i],
                self.label_binarizer_.classes_[i]])
                                                        for i in range(len(YBal)))
        # for i, column in enumerate(columns))
        return self

class HeurFeat():
    def heur_c_k_001(self,input_text):
        if ('report bugs' in input_text.lower()) or ('reporting bugs' in input_text.lower()):
            return 1
        else:
            return 0

    def heur_c_k_002(self,input_text):
        if ' is a ' in input_text.lower():
            return 1
        else:
            return 0

    def heur_c_k_003(self,input_text):
        if '@abstr_code_section' in input_text.lower():
            return 1
        else:
            return 0

    def heur_c_k_004(self,input_text):
        if 'attempts to' in input_text.lower():
            return 1
        else:
            return 0

    def heur_c_k_005(self,input_text):
        if 'inspired by' in input_text.lower():
            return 1
        else:
            return 0

    def heur_c_k_006(self,input_text):
        if 'install ' in input_text.lower():
            return 1
        else:
            return 0

    def heur_c_k_007(self,input_text):
        if 'reasons' in input_text.lower():
            return 1
        else:
            return 0

    # Do not use lower() because we want to capture "Added" at beginning of sentence
    def heur_c_k_008(self,input_text):
        if 'Added ' in input_text:
            return 1
        else:
            return 0

    def heur_c_k_009(self,input_text):
        if 'copyright' in input_text.lower():
            return 1
        else:
            return 0

    def heur_c_k_010(self,input_text):
        if '@abstr_mailto' in input_text.lower():
            return 1
        else:
            return 0

    def heur_c_k_011(self,input_text):
        if 'you can ' in input_text.lower():
            return 1
        else:
            return 0

    # Check if the text comprises solely of @abstr_code_section
    def heur_c_k_012(self,input_text):
        if '@abstr_code_section' == input_text.lower().strip():
            return 1
        else:
            return 0

    def heur_c_k_013(self,input_text):
        if 'About' in input_text:
            return 1
        else:
            return 0

    def heur_c_k_014(self,input_text):
        if 'be sure to' in input_text.lower():
            return 1
        else:
            return 0

    def heur_c_k_015(self,input_text):
        if 'Download' in input_text:
            return 1
        else:
            return 0

    def heur_c_k_016(self,input_text):
        if 'overview' in input_text.lower():
            return 1
        else:
            return 0

    def heur_c_k_017(self,input_text):
        if 'get started' in input_text.lower():
            return 1
        else:
            return 0

    def heur_c_k_018(self,input_text):
        if 'reasons' in input_text.lower():
            return 1
        else:
            return 0

    def heur_c_k_019(self,input_text):
        if 'dependenc' in input_text.lower():
            return 1
        else:
            return 0

    def heur_c_k_020(self,input_text):
        if 'rerun' in input_text.lower():
            return 1
        else:
            return 0

    def heur_c_k_021(self,input_text):
        if 'you''ll be able' in input_text.lower():
            return 1
        else:
            return 0

    def heur_c_k_022(self,input_text):
        if 'you must' in input_text.lower():
            return 1
        else:
            return 0

    def heur_c_k_023(self,input_text):
        if 'previous version' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_001(self,input_text):
        if 'configur' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_002(self,input_text):
        if 'what' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_003(self,input_text):
        if 'why' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_004(self,input_text):
        if 'approach' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_005(self,input_text):
        if 'bugs' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_006(self,input_text):
        if 'contrib' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_007(self,input_text):
        if 'credit' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_008(self,input_text):
        if 'feature' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_009(self,input_text):
        if 'install' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_010(self,input_text):
        if 'intro' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_011(self,input_text):
        if 'licen' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_012(self,input_text):
        if 'objective' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_013(self,input_text):
        if 'request' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_014(self,input_text):
        if 'requirement' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_015(self,input_text):
        if 'resource' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_016(self,input_text):
        if 'setting' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_017(self,input_text):
        if 'setup' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_018(self,input_text):
        if 'started' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_019(self,input_text):
        if 'usage' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_020(self,input_text):
        if 'version' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_021(self,input_text):
        if 'welcome' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_022(self,input_text):
        if 'what is' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_023(self,input_text):
        if 'overview' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_024(self,input_text):
        if 'basic' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_025(self,input_text):
        if 'roadmap' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_026(self,input_text):
        if 'todo' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_027(self,input_text):
        if 'example' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_028(self,input_text):
        if 'about' in input_text.lower():
            return 1
        else:
            return 0

    def heur_h_k_029(self,input_text):
        if 'reference' in input_text.lower():
            return 1
        else:
            return 0

            # Return 1 if string is single non-English word

    # WARNING: For speed, instantiate the word set outside and pass it as parameter,
    # so that instantiation will be done once for each program run
    # instead of once for each row this heuristic is applied to
    def heur_h_c_001(self,input_text, words=None):
        nltk.download('words')
        lcase_input_text = input_text.lower()
        s = lcase_input_text.split(' ')
        if len(s) != 1:
            return 0
        else:
            if words is None:
                words = set(nltk.corpus.words.words())
            if s[0] in words:
                return 0
            else:
                return 1
        return 0

    # Returns 1 if a word in heading text is in repo name, or the other way around
    def heur_h_c_002(self,heading_text, repo_url):
        nltk.download('words')
        heading_words = heading_text.lower().split(' ')
        repo_name = re.sub(r"http(s)*:\/\/www\.github\.com\/.+\/", '', repo_url)
        repo_name = repo_name.replace('.', ' ').replace('-', ' ')
        repo_name_words = repo_name.lower().split(' ')
        match = [val for val in heading_words if val in repo_name_words]
        if len(match) > 0:
            return 1
        else:
            return 0

    # See whether text comprises entirely of ASCII characters
    def heur_c_s_001(self,input_text):
        try:
            input_text.encode(encoding='utf-8').decode('ascii')
        except UnicodeDecodeError:
            return 0
        else:
            return 1

            # Generate DataFrame of derived features using DataFrames of some initial features

    def derive_features_using_heuristics(self,url_corpus, heading_text_corpus, content_corpus):
        nltk.download('words')
        derived_features = DataFrame()
        derived_features['heur_c_k_001'] = [self.heur_c_k_001(x) for x in content_corpus]
        derived_features['heur_c_k_002'] = [self.heur_c_k_002(x) for x in content_corpus]
        derived_features['heur_c_k_003'] = [self.heur_c_k_003(x) for x in content_corpus]
        derived_features['heur_c_k_004'] = [self.heur_c_k_004(x) for x in content_corpus]
        derived_features['heur_c_k_005'] = [self.heur_c_k_005(x) for x in content_corpus]
        derived_features['heur_c_k_006'] = [self.heur_c_k_006(x) for x in content_corpus]
        derived_features['heur_c_k_007'] = [self.heur_c_k_007(x) for x in content_corpus]
        derived_features['heur_h_k_001'] = [self.heur_h_k_001(x) for x in heading_text_corpus]
        derived_features['heur_h_k_002'] = [self.heur_h_k_002(x) for x in heading_text_corpus]
        derived_features['heur_h_k_003'] = [self.heur_h_k_003(x) for x in heading_text_corpus]
        derived_features['heur_h_k_004'] = [self.heur_h_k_004(x) for x in heading_text_corpus]
        derived_features['heur_h_k_005'] = [self.heur_h_k_005(x) for x in heading_text_corpus]
        derived_features['heur_h_k_006'] = [self.heur_h_k_006(x) for x in heading_text_corpus]
        derived_features['heur_h_k_007'] = [self.heur_h_k_007(x) for x in heading_text_corpus]
        derived_features['heur_h_k_008'] = [self.heur_h_k_008(x) for x in heading_text_corpus]
        derived_features['heur_h_k_009'] = [self.heur_h_k_009(x) for x in heading_text_corpus]
        derived_features['heur_h_k_010'] = [self.heur_h_k_010(x) for x in heading_text_corpus]
        derived_features['heur_h_k_011'] = [self.heur_h_k_011(x) for x in heading_text_corpus]
        derived_features['heur_h_k_012'] = [self.heur_h_k_012(x) for x in heading_text_corpus]
        derived_features['heur_h_k_013'] = [self.heur_h_k_013(x) for x in heading_text_corpus]
        derived_features['heur_h_k_014'] = [self.heur_h_k_014(x) for x in heading_text_corpus]
        derived_features['heur_h_k_015'] = [self.heur_h_k_015(x) for x in heading_text_corpus]
        derived_features['heur_h_k_016'] = [self.heur_h_k_016(x) for x in heading_text_corpus]
        derived_features['heur_h_k_017'] = [self.heur_h_k_017(x) for x in heading_text_corpus]
        derived_features['heur_h_k_018'] = [self.heur_h_k_018(x) for x in heading_text_corpus]
        derived_features['heur_h_k_019'] = [self.heur_h_k_019(x) for x in heading_text_corpus]
        derived_features['heur_h_k_020'] = [self.heur_h_k_020(x) for x in heading_text_corpus]
        derived_features['heur_h_k_021'] = [self.heur_h_k_021(x) for x in heading_text_corpus]
        derived_features['heur_h_k_022'] = [self.heur_h_k_022(x) for x in heading_text_corpus]
        derived_features['heur_c_s_001'] = [self.heur_c_s_001(x) for x in heading_text_corpus]

        # Batch 02
        derived_features['heur_c_k_008'] = [self.heur_c_k_008(x) for x in content_corpus]
        derived_features['heur_c_k_009'] = [self.heur_c_k_009(x) for x in content_corpus]
        derived_features['heur_c_k_010'] = [self.heur_c_k_010(x) for x in content_corpus]
        derived_features['heur_c_k_011'] = [self.heur_c_k_011(x) for x in content_corpus]
        derived_features['heur_c_k_012'] = [self.heur_c_k_012(x) for x in content_corpus]
        derived_features['heur_c_k_013'] = [self.heur_c_k_013(x) for x in content_corpus]
        derived_features['heur_h_k_023'] = [self.heur_h_k_023(x) for x in heading_text_corpus]
        derived_features['heur_h_k_024'] = [self.heur_h_k_024(x) for x in heading_text_corpus]
        derived_features['heur_h_k_025'] = [self.heur_h_k_025(x) for x in heading_text_corpus]
        derived_features['heur_h_k_026'] = [self.heur_h_k_026(x) for x in heading_text_corpus]
        derived_features['heur_h_k_027'] = [self.heur_h_k_027(x) for x in heading_text_corpus]
        words = set(nltk.corpus.words.words())
        derived_features['heur_h_c_001'] = [self.heur_h_c_001(x, words) for x in heading_text_corpus]
        derived_features['heur_h_c_002'] = [self.heur_h_c_002(x, y) for x, y in zip(heading_text_corpus, url_corpus)]

        # Batch 03
        derived_features['heur_c_k_014'] = [self.heur_c_k_014(x) for x in content_corpus]
        derived_features['heur_c_k_015'] = [self.heur_c_k_015(x) for x in content_corpus]
        derived_features['heur_c_k_016'] = [self.heur_c_k_016(x) for x in content_corpus]
        derived_features['heur_c_k_017'] = [self.heur_c_k_017(x) for x in content_corpus]
        derived_features['heur_c_k_018'] = [self.heur_c_k_018(x) for x in content_corpus]
        derived_features['heur_c_k_019'] = [self.heur_c_k_019(x) for x in content_corpus]
        derived_features['heur_c_k_020'] = [self.heur_c_k_020(x) for x in content_corpus]
        derived_features['heur_c_k_021'] = [self.heur_c_k_021(x) for x in content_corpus]
        derived_features['heur_c_k_022'] = [self.heur_c_k_022(x) for x in content_corpus]
        derived_features['heur_c_k_023'] = [self.heur_c_k_023(x) for x in content_corpus]
        derived_features['heur_h_k_028'] = [self.heur_h_k_028(x) for x in heading_text_corpus]
        derived_features['heur_h_k_029'] = [self.heur_h_k_029(x) for x in heading_text_corpus]

        return derived_features

In [0]:
class run_algorithms():
    df = None
    db_filename = "../database/data.db"
    rng_seed = 100
    log_filename = '../log/experiment_classifier_validation.log'

    

    def __init__(self):
        
        self.analysis_preparation()



    def analysis_preparation(self):
        
        self.load_df_csv()

        self.prepare_data()
        print("Ready for classify. Run test_classifier with model of your choice.")

    def load_df_csv(self):
        df_python = (
        sqlContext
        .read.format("csv")
        .option("header", "true")
    
        .load('/FileStore/FinalProject/old_and_new_data-1.csv')
          )
        self.df = df_python.toPandas()

    def test_classifier(self, classifier):
        ca = OneVsRestClassifierBalance(classifier)
        print(f'Running experiment for {classifier}')
        print('Getting per-class scores')


        logging.info(f"labels matrix type: {type(self.labels_matrix)}")
        logging.info(f"labels matrix shape: {self.labels_matrix.shape}")
        y_pred = cross_val_predict(ca, self.features_combined.values, self.labels_matrix, cv=10)

        print('Computing overall results')
        scores_f1 = cross_val_score(classifier, self.features_combined.values, self.labels_matrix, cv=10,
                                    scoring='f1_weighted').mean()

        print(classification_report(self.labels_matrix, y_pred, digits=3))
        print('f1_weighted : {0}'.format(scores_f1))

    def load_df_sql(self):
        try:
            sql_text = """
                SELECT t1.file_id, t1.section_id, t1.url, t1.heading_text, t2.content_text_w_o_tags, 
                t1.abstracted_heading_text || ' ' || t2.content_text_w_o_tags AS abstracted_heading_plus_content, 
                t1.section_code
                FROM section_overview_75pct t1 
                JOIN section_content_75pct t2 ON t1.file_id=t2.file_id AND t1.section_id=t2.section_id
                """
            self.df = pandas.read_sql_query(con=self.conn, sql=sql_text)
        except Error as e:
            logging.exception(e)
        except Exception as e:
            logging.exception(e)
        finally:
            self.conn.close()

    def prepare_data(self):
        df_randomized_order = self.df.sample(frac=1, random_state=self.rng_seed)
        heading_plus_content_corpus = df_randomized_order['abstracted_heading_plus_content']
        content_corpus = df_randomized_order['content_text_w_o_tags']
        heading_text_corpus = df_randomized_order['heading_text']
        url_corpus = df_randomized_order['url']
        label_set = ['-', '1', '3', '4', '5', '6', '7', '8']
        labels = [str(x).split(',') for x in df_randomized_order['section_code']]
        mlb = MultiLabelBinarizer(classes=label_set)
        self.labels_matrix = mlb.fit_transform(labels)

        tfidf = TfidfVectorizer(ngram_range=(1, 1), analyzer='word', stop_words='english')
        tfidfX = tfidf.fit_transform(heading_plus_content_corpus)
        ca = HeurFeat()
        derived_features = ca.derive_features_using_heuristics(url_corpus, heading_text_corpus, content_corpus)
        features_tfidf = pandas.DataFrame(tfidfX.todense())
        # Assign column names to make it easier to print most useful features later
        features_tfidf.columns = tfidf.get_feature_names()
        self.features_combined = pandas.concat([features_tfidf, derived_features], axis=1)

In [0]:
starter = run_algorithms()
pandas.set_option('display.max_columns', None)
print(starter.df.head())


/databricks/python/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:860: UserWarning: unknown class(es) ['                [""[a-f @abstr_number - @abstr_number ]+""', '         ""mail"" : ""sabre+ @abstr_number @tut.by""         ""role"" : ""Store Owner""       }        -- response --       Set-Cookie:  SESSe @abstr_number a @abstr_number a @abstr_number c @abstr_number a @abstr_number c @abstr_number b @abstr_number cb @abstr_number f @abstr_number =E @abstr_number juSPMd @abstr_number fUOg @abstr_number j_esS @abstr_number G @abstr_number MwU @abstr_number jodrDpBjl @abstr_number KHfli @abstr_number ; expires=Sat', "         forro = require('forro')", '         location: ""YOUR_LOCATION""', '     ""express"": ""^ @abstr_number . @abstr_number . @abstr_number ""', '     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND', '     to deal in the Software without restriction', '   * @abstr_hyperlink ', '   * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND', '   * an @abstr_hyperlink -ba

In [0]:
starter.test_classifier(RandomForestClassifier())

Running experiment for RandomForestClassifier()
Getting per-class scores
Computing overall results
              precision    recall  f1-score   support

           0      0.513     0.744     0.607       422
           1      0.748     0.400     0.521       652
           2      0.814     0.857     0.835      2234
           3      0.882     0.444     0.590       151
           4      0.902     0.540     0.676       274
           5      0.720     0.502     0.592       811
           6      0.919     0.528     0.671       108
           7      0.119     0.623     0.201        61

   micro avg      0.713     0.680     0.696      4713
   macro avg      0.702     0.580     0.587      4713
weighted avg      0.763     0.680     0.700      4713
 samples avg      0.655     0.664     0.651      4713

f1_weighted : 0.6308957564173909
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set

In [0]:
starter.test_classifier(LinearSVC())

Running experiment for LinearSVC()
Getting per-class scores
Computing overall results
/databricks/python/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/databricks/python/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_sklearn.py", line 29, in patch_function
    original_result = original(self, *args, **kwargs)
  File "/databricks/python/lib/python3.8/site-packages/sklearn/svm/_classes.py", line 228, in fit
    X, y = self._validate_data(X, y, accept_sparse='csr',
  File "/databricks/python/lib/python3.8/site-packages/sklearn/base.py", line 433, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/databricks

In [0]:
starter.test_classifier(GaussianNB())

Running experiment for GaussianNB()
Getting per-class scores
Computing overall results
/databricks/python/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/databricks/python/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_sklearn.py", line 29, in patch_function
    original_result = original(self, *args, **kwargs)
  File "/databricks/python/lib/python3.8/site-packages/sklearn/naive_bayes.py", line 207, in fit
    X, y = self._validate_data(X, y)
  File "/databricks/python/lib/python3.8/site-packages/sklearn/base.py", line 433, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/databricks/python/lib/python3.8

In [0]:
starter.test_classifier(LogisticRegression())

Running experiment for LogisticRegression()
Getting per-class scores
Computing overall results
/databricks/python/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/databricks/python/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_sklearn.py", line 29, in patch_function
    original_result = original(self, *args, **kwargs)
  File "/databricks/python/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1344, in fit
    X, y = self._validate_data(X, y, accept_sparse='csr', dtype=_dtype,
  File "/databricks/python/lib/python3.8/site-packages/sklearn/base.py", line 433, in _validate_data
    X, y = check_X_y(X, y, *

In [0]:
starter.test_classifier(KNeighborsClassifier())

Running experiment for KNeighborsClassifier()
Getting per-class scores
Computing overall results
              precision    recall  f1-score   support

           0      0.182     0.874     0.301       422
           1      0.474     0.468     0.471       652
           2      0.818     0.620     0.705      2234
           3      0.421     0.656     0.513       151
           4      0.585     0.540     0.562       274
           5      0.369     0.420     0.393       811
           6      0.661     0.722     0.690       108
           7      0.032     0.869     0.063        61

   micro avg      0.369     0.589     0.454      4713
   macro avg      0.443     0.646     0.462      4713
weighted avg      0.596     0.589     0.560      4713
 samples avg      0.471     0.576     0.499      4713

f1_weighted : 0.5253375257247371
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set t